In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

2024-11-03 18:33:43.966671: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-03 18:33:44.066405: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-03 18:33:44.591520: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-11-03 18:33:44.591576: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

GPU is available


In [2]:
# Read from CSV instead of pickle
dataset = pd.read_csv('cleaned_profiles.csv')
dataset

,Intro,Full Name,Workplace,Location,Connections,Photo,Followers,About,Experiences,Number of Experiences,...,Number of Scores,Languages,Number of Languages,Organizations,Number of Organizations,Interests,Number of Interests,Activities,Number of Activities,Label
0,"{'Full Name': 'chenxia (polly) Pei', 'Workplac...",chenxia (polly) Pei,Jiangsu Junyao mainly offer services to cement...,"Wuxi, Jiangsu, China",500,No,717,NaN,"{'0': {'Role': 'Sales', 'Workplace': 'Jiangsu ...",2,...,0,{},0,{},0,"{'0': {'Interest': 'Trina Solar', 'ID': '69648...",4,{'chenxia-pei-80177594': {'Full Name': 'chenxi...,1,1
1,"{'Full Name': 'NEHA CHANDOK', 'Workplace': 'So...",NEHA CHANDOK,Software Analyst,"Noida, Uttar Pradesh, India",500,No,1340,NaN,"{'0': {'Role': 'Software Analyst', 'Workplace'...",1,...,0,"{'Language 0': 'Bangla', 'Language 1': 'Englis...",3,{},0,"{'0': {'Interest': 'Mohamed El-Erian', 'ID': '...",8,{},0,1
2,"{'Full Name': 'Mounika Mungamuri', 'Workplace'...",Mounika Mungamuri,Senior Consultant at Infosys,"Hyderabad, Telangana, India",7,Yes,7,NaN,"{'0': {'Role': 'Senior Consultant', 'Workplace...",2,...,0,{},0,{},0,{},0,{},0,1
3,"{'Full Name': 'Katarina Djuric', 'Workplace': ...",Katarina Djuric,--,"Belgrade, Serbia",0,Yes,0,NaN,"{'0': {'Role': 'Instructor', 'Workplace': 'GE'...",1,...,0,{},0,{},0,"{'0': {'Interest': 'GE', 'ID': '1015', 'Catego...",1,{},0,1
4,"{'Full Name': 'Rachel Lally', 'Workplace': '--...",Rachel Lally,--,"Dublin, County Dublin, Ireland",61,Yes,61,NaN,"{'0': {'Role': 'Bartender', 'Workplace': ""O'Su...",1,...,1,{},0,{},0,"{'0': {'Interest': 'Richard Branson', 'ID': 'r...",3,"{'garyltravis': {'Full Name': 'Gary Travis', '...",6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,"{'Full Name': 'Emily Williams', 'Workplace': ""...",Emily Williams,Market Research Analyst at L'Oreal,"New York City, New York",106,No,717,{},Market Research Analyst Elizabeth Arden Jan 20...,3,...,0,{},0,{},0,{},3,{},4,11
3596,"{'Full Name': 'Sarah Johnson', 'Workplace': 'D...",Sarah Johnson,Director of Marketing Strategy at Acme Inc.,"San Francisco, California",102,No,6,{},Director of Marketing Acme Inc. January 2018-P...,5,...,0,{},0,{},0,{},5,{},3,11
3597,"{'Full Name': 'Sarah Johnson', 'Workplace': 'M...",Sarah Johnson,Manager at Acme Inc.,"New York City, New York",435,No,10,A results-driven and experienced professional ...,{},0,...,0,{},0,{},0,{},6,{},0,11
3598,"{'Full Name': 'Emma Williams', 'Workplace': 'M...",Emma Williams,Manager and Director at ABC Inc.,"Seattle, Washington",280,No,34,With over 10 years of experience in operations...,Operations Manager ABC Inc. January 2015- Pres...,1,...,0,English Spanish,2,{},0,{},4,{},4,11


In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# Load the BERT embeddings
emb = pd.read_csv('bert_numerical_STE_tokenised_D_NEW.csv')

# Prepare the features (X) and labels (y)
X = emb
dataset.loc[dataset['Label'].isin([10, 11]), 'Label'] = 1
y = dataset["Label"]

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

# Function to train and evaluate classifiers
def train_and_evaluate_classifier(X, y, classifier, test_size=0.3, random_state=42):
    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=360, train_size=840, stratify=y, random_state=42)
    
    # Initialize the classifier based on the input string
    if classifier == 'random_forest':
        clf = RandomForestClassifier(random_state=random_state)
    elif classifier == 'logistic_regression':
        clf = LogisticRegression(random_state=random_state)
    elif classifier == 'svc':
        clf = SVC(random_state=random_state)
    elif classifier == 'knn':
        clf = KNeighborsClassifier()
    elif classifier == 'xgboost':
        clf = XGBClassifier(random_state=random_state)
    elif classifier == 'catboost':
        clf = CatBoostClassifier(random_state=random_state, verbose=False)
    else:
        raise ValueError("Invalid classifier type.")

    # Train the classifier on the training data
    clf.fit(X_train, y_train)

    # Predict the labels for the test set
    y_pred = clf.predict(X_test)

    # Calculate the accuracy of the classifier
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

# List of classifiers to test
classifiers = ['random_forest', 'logistic_regression', 'svc', 'knn', 'xgboost', 'catboost']

# Set a common random state for reproducibility
random_state = 42

# Iterate over each classifier and call the function
for classifier in classifiers:
    accuracy = train_and_evaluate_classifier(X, y, classifier, random_state=random_state)
    print(f"Accuracy of {classifier.capitalize()} Classifier: {accuracy:.4f}")

# Print the number of samples used for training and testing
print(f"\nNumber of training samples: 840")
print(f"Number of testing samples: 360")
print(f"Total samples: 1200")

Shape of X: (3600, 768)
Shape of y: (3600,)
Accuracy of Random_forest Classifier: 0.9389
Accuracy of Logistic_regression Classifier: 0.9083
Accuracy of Svc Classifier: 0.8500
Accuracy of Knn Classifier: 0.9250
Accuracy of Xgboost Classifier: 0.9472
Accuracy of Catboost Classifier: 0.9444

Number of training samples: 840
Number of testing samples: 360
Total samples: 1200


In [39]:
emb = pd.read_csv('bert_numerical_STE_tokenised_D_NEW.csv')

# Prepare the features (X) and labels (y)
X = emb
dataset.loc[dataset['Label'].isin([10, 11]), 'Label'] = 1
y = dataset["Label"]

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (3600, 768)
Shape of y: (3600,)


In [40]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
import optuna
from deap import base, creator, tools, algorithms
import random
import warnings
from datetime import datetime

warnings.filterwarnings("ignore")

def prepare_data(X, y, random_state=42):
    print(f"\nPreparing data...")
    print(f"Input shape: X={X.shape}, y={y.shape}")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=360, train_size=840, stratify=y, random_state=random_state)
    print(f"After split: X_train={X_train.shape}, X_test={X_test.shape}")
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    print("Data scaling completed")
    return X_train_scaled, X_test_scaled, y_train, y_test

# Genetic Algorithm
def genetic_algorithm(X, y, random_state=42):
    start_time = datetime.now()
    print(f"\nStarting Genetic Algorithm at {start_time.strftime('%H:%M:%S')}")
    X_train, X_test, y_train, y_test = prepare_data(X, y, random_state)

    def evaluate(individual):
        params = {
            'depth': max(1, individual[0]),
            'learning_rate': max(0.001, individual[1]),
            'iterations': max(10, individual[2]),  
            'l2_leaf_reg': max(0, individual[3]),
            'border_count': max(32, individual[4]),
            'verbose': False
        }
        clf = CatBoostClassifier(**params, random_state=random_state)
        clf.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=20, verbose=False)
        y_pred = clf.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        return acc,

    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)

    toolbox = base.Toolbox()
    toolbox.register("attr_depth", random.randint, 1, 10)
    toolbox.register("attr_learning_rate", random.uniform, 0.001, 0.3)
    toolbox.register("attr_iterations", random.randint, 10, 1000)
    toolbox.register("attr_l2_leaf_reg", random.uniform, 0, 10)
    toolbox.register("attr_border_count", random.randint, 32, 255)

    toolbox.register("individual", tools.initCycle, creator.Individual,
                     (toolbox.attr_depth, toolbox.attr_learning_rate, toolbox.attr_iterations,
                      toolbox.attr_l2_leaf_reg, toolbox.attr_border_count), n=1)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    toolbox.register("evaluate", evaluate)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    population = toolbox.population(n=50)
    ngen = 10

    # Add statistics for tracking progress
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)
    stats.register("std", np.std)

    print(f"\nGA Configuration:")
    print(f"Population size: {len(population)}")
    print(f"Number of generations: {ngen}")
    print(f"Starting evolution...")

    population, logbook = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, 
                                            ngen=ngen, stats=stats, verbose=True)

    best_individual = tools.selBest(population, k=1)[0]
    best_params = {
        'depth': max(1, best_individual[0]),
        'learning_rate': max(0.001, best_individual[1]),
        'iterations': max(10, best_individual[2]),
        'l2_leaf_reg': max(0, best_individual[3]),
        'border_count': max(32, best_individual[4])
    }
    best_accuracy = best_individual.fitness.values[0]
    
    end_time = datetime.now()
    duration = end_time - start_time
    print(f"\nGA completed at {end_time.strftime('%H:%M:%S')}")
    print(f"Total duration: {duration}")
    return best_params, best_accuracy

# Bayesian Optimization
def bayesian_optimization(X, y, random_state=42):
    start_time = datetime.now()
    print(f"\nStarting Bayesian Optimization at {start_time.strftime('%H:%M:%S')}")
    X_train, X_test, y_train, y_test = prepare_data(X, y, random_state)
    
    best_accuracy_so_far = 0
    trial_times = []

    def objective(trial):
        nonlocal best_accuracy_so_far
        trial_start = datetime.now()
        print(f"\nTrial {trial.number + 1}/50 started at {trial_start.strftime('%H:%M:%S')}")
        
        params = {
            'depth': trial.suggest_int('depth', 1, 10),
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.3),
            'iterations': trial.suggest_int('iterations', 10, 1000),
            'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 0.00001, 10),
            'border_count': trial.suggest_int('border_count', 32, 255),
            'verbose': False
        }
        clf = CatBoostClassifier(**params, random_state=random_state)
        clf.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=20, verbose=False)
        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        
        trial_end = datetime.now()
        trial_duration = trial_end - trial_start
        trial_times.append(trial_duration.total_seconds())
        avg_trial_time = np.mean(trial_times)
        
        best_accuracy_so_far = max(best_accuracy_so_far, accuracy)
        print(f"Trial {trial.number + 1} results:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Best accuracy so far: {best_accuracy_so_far:.4f}")
        print(f"Trial duration: {trial_duration}")
        print(f"Average trial duration: {avg_trial_time:.2f} seconds")
        print(f"Estimated time remaining: {avg_trial_time * (49 - trial.number):.2f} seconds")
        
        return accuracy

    study = optuna.create_study(direction='maximize')
    print(f"Running Bayesian Optimization for 50 trials...")
    study.optimize(objective, n_trials=50)
    
    end_time = datetime.now()
    duration = end_time - start_time
    print(f"\nBayesian Optimization completed at {end_time.strftime('%H:%M:%S')}")
    print(f"Total duration: {duration}")
    return study.best_params, study.best_value

# Main execution
random_state = 42

print(f"Starting hyperparameter optimization at {datetime.now().strftime('%H:%M:%S')}")
print(f"Number of training samples: 840")
print(f"Number of testing samples: 360")
print(f"Total samples: 1200")

print("\nGenetic Algorithm Optimization")
ga_start = datetime.now()
ga_best_params, ga_best_accuracy = genetic_algorithm(X, y, random_state=random_state)
print("\nBest parameters (Genetic Algorithm):", ga_best_params)
print("Best accuracy (Genetic Algorithm):", ga_best_accuracy)

print("\nBayesian Optimization")
bo_start = datetime.now()
bo_best_params, bo_best_accuracy = bayesian_optimization(X, y, random_state=random_state)
print("\nBest parameters (Bayesian Optimization):", bo_best_params)
print("Best accuracy (Bayesian Optimization):", bo_best_accuracy)

total_duration = datetime.now() - ga_start
print(f"\nTotal optimization time: {total_duration}")

Starting hyperparameter optimization at 00:41:59
Number of training samples: 840
Number of testing samples: 360
Total samples: 1200

Genetic Algorithm Optimization

Starting Genetic Algorithm at 00:41:59

Preparing data...
Input shape: X=(3600, 768), y=(3600,)
After split: X_train=(840, 768), X_test=(360, 768)
Data scaling completed

GA Configuration:
Population size: 50
Number of generations: 10
Starting evolution...
gen	nevals	avg     	min     	max     	std      
0  	50    	0.943444	0.922222	0.963889	0.0112046
1  	33    	0.951389	0.922222	0.963889	0.0093994
2  	33    	0.954889	0.936111	0.963889	0.00639637
3  	24    	0.958722	0.944444	0.963889	0.00535787
4  	39    	0.960167	0.941667	0.963889	0.00495691
5  	27    	0.962111	0.941667	0.963889	0.00447076
6  	28    	0.963222	0.955556	0.963889	0.00196889
7  	33    	0.961667	0.9     	0.963889	0.00936239
8  	30    	0.961889	0.888889	0.963889	0.0105853 
9  	35    	0.963889	0.963889	0.963889	0         


[I 2024-11-04 00:47:53,851] A new study created in memory with name: no-name-9365dc07-914c-4f0b-9d66-fe257dc24c31


10 	31    	0.963722	0.955556	0.963889	0.00116667

GA completed at 00:47:53
Total duration: 0:05:54.273232

Best parameters (Genetic Algorithm): {'depth': 1, 'learning_rate': 0.18367011746081183, 'iterations': 489, 'l2_leaf_reg': 5.903845378061958, 'border_count': 204}
Best accuracy (Genetic Algorithm): 0.9638888888888889

Bayesian Optimization

Starting Bayesian Optimization at 00:47:53

Preparing data...
Input shape: X=(3600, 768), y=(3600,)
After split: X_train=(840, 768), X_test=(360, 768)
Data scaling completed
Running Bayesian Optimization for 50 trials...

Trial 1/50 started at 00:47:53


[I 2024-11-04 00:47:54,644] Trial 0 finished with value: 0.9444444444444444 and parameters: {'depth': 6, 'learning_rate': 0.012497741501685387, 'iterations': 64, 'l2_leaf_reg': 0.0002910376589038683, 'border_count': 69}. Best is trial 0 with value: 0.9444444444444444.


Trial 1 results:
Accuracy: 0.9444
Best accuracy so far: 0.9444
Trial duration: 0:00:00.790581
Average trial duration: 0.79 seconds
Estimated time remaining: 38.74 seconds

Trial 2/50 started at 00:47:54


...

Trial 48 results:
Accuracy: 0.9556
Best accuracy so far: 0.9611
Trial duration: 0:00:00.825335
Average trial duration: 2.22 seconds
Estimated time remaining: 4.44 seconds

Trial 49/50 started at 00:49:40


[I 2024-11-04 00:49:42,530] Trial 48 finished with value: 0.95 and parameters: {'depth': 2, 'learning_rate': 0.022011216982598867, 'iterations': 515, 'l2_leaf_reg': 0.5629918401898496, 'border_count': 123}. Best is trial 33 with value: 0.9611111111111111.


Trial 49 results:
Accuracy: 0.9500
Best accuracy so far: 0.9611
Trial duration: 0:00:01.948191
Average trial duration: 2.22 seconds
Estimated time remaining: 2.22 seconds

Trial 50/50 started at 00:49:42


[I 2024-11-04 00:49:43,042] Trial 49 finished with value: 0.9472222222222222 and parameters: {'depth': 3, 'learning_rate': 0.14947645887702457, 'iterations': 420, 'l2_leaf_reg': 0.08007217434813806, 'border_count': 186}. Best is trial 33 with value: 0.9611111111111111.


Trial 50 results:
Accuracy: 0.9472
Best accuracy so far: 0.9611
Trial duration: 0:00:00.510816
Average trial duration: 2.18 seconds
Estimated time remaining: 0.00 seconds

Bayesian Optimization completed at 00:49:43
Total duration: 0:01:49.219972

Best parameters (Bayesian Optimization): {'depth': 2, 'learning_rate': 0.2044854959627784, 'iterations': 520, 'l2_leaf_reg': 0.0472394887803851, 'border_count': 168}
Best accuracy (Bayesian Optimization): 0.9611111111111111

Total optimization time: 0:07:43.496803


In [41]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from deap import base, creator, tools, algorithms
import optuna
import random
import time
import warnings
import torch

warnings.filterwarnings("ignore")

# Define the parameter spaces for XGBoost and CatBoost
param_space = {
    'xgboost': {
        'max_depth': list(range(3, 11)),
        'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
        'n_estimators': list(range(50, 551, 50)),
        'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    },
    'catboost': {
        'depth': list(range(4, 11)),
        'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
        'iterations': list(range(50, 551, 50)),
        'l2_leaf_reg': [1, 3, 5, 7, 9],
        'border_count': [32, 64, 128, 254]
    }
}

def prepare_data(X, y, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=360, train_size=840, stratify=y, random_state=random_state)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, y_train, y_test

def genetic_algorithm(X, y, classifier_type, random_state=42):
    X_train_scaled, X_test_scaled, y_train, y_test = prepare_data(X, y, random_state)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def evaluate(individual):
        if classifier_type == 'xgboost':
            clf = XGBClassifier(
                max_depth=individual[0],
                learning_rate=individual[1],
                n_estimators=individual[2],
                subsample=min(max(individual[3], 0.6), 1.0),  # Ensure subsample is between 0.6 and 1.0
                colsample_bytree=min(max(individual[4], 0.6), 1.0),  # Ensure colsample_bytree is between 0.6 and 1.0
                tree_method='gpu_hist' if device == 'cuda' else 'hist',
                random_state=random_state,
                early_stopping_rounds=10,
                eval_metric='logloss'
            )
            clf.fit(X_train_scaled, y_train, eval_set=[(X_test_scaled, y_test)], verbose=False)
        else:  # CatBoost
            clf = CatBoostClassifier(
                depth=individual[0],
                learning_rate=individual[1],
                iterations=individual[2],
                l2_leaf_reg=individual[3],
                border_count=individual[4],
                task_type='GPU' if device == 'cuda' else 'CPU',
                random_state=random_state,
                verbose=False
            )
            clf.fit(X_train_scaled, y_train, eval_set=(X_test_scaled, y_test), 
                    early_stopping_rounds=10, verbose=False)
        
        y_pred = clf.predict(X_test_scaled)
        return accuracy_score(y_test, y_pred),

    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)
    toolbox = base.Toolbox()

    param_keys = list(param_space[classifier_type].keys())
    for i, key in enumerate(param_keys):
        toolbox.register(f"attr_{i}", random.choice, param_space[classifier_type][key])

    toolbox.register("individual", tools.initCycle, creator.Individual,
                     [getattr(toolbox, f"attr_{i}") for i in range(len(param_keys))], n=1)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", evaluate)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutUniformInt, low=0, up=10, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)

    population = toolbox.population(n=50)
    ngen = 10

    start_time = time.time()
    print(f"Starting Genetic Algorithm optimization for {classifier_type.upper()}...")

    result, log = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=ngen, verbose=True)

    end_time = time.time()
    print(f"Finished Genetic Algorithm tuning. Time taken: {end_time - start_time:.2f} seconds")

    best_individual = tools.selBest(result, k=1)[0]
    best_params = dict(zip(param_keys, best_individual))
    if classifier_type == 'xgboost':
        best_params['subsample'] = min(max(best_params['subsample'], 0.6), 1.0)
        best_params['colsample_bytree'] = min(max(best_params['colsample_bytree'], 0.6), 1.0)
    best_accuracy = best_individual.fitness.values[0]

    return best_params, best_accuracy

def bayesian_optimization(X, y, classifier_type, random_state=42):
    X_train_scaled, X_test_scaled, y_train, y_test = prepare_data(X, y, random_state)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def objective(trial):
        if classifier_type == 'xgboost':
            params = {
                'max_depth': trial.suggest_int('max_depth', 3, 10),
                'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
                'n_estimators': trial.suggest_int('n_estimators', 50, 550),
                'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
                'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
                'tree_method': 'gpu_hist' if device == 'cuda' else 'hist',
                'random_state': random_state,
                'early_stopping_rounds': 10,
                'eval_metric': 'logloss'
            }
            clf = XGBClassifier(**params)
            clf.fit(X_train_scaled, y_train, eval_set=[(X_test_scaled, y_test)], verbose=False)
        else:  # CatBoost
            params = {
                'depth': trial.suggest_int('depth', 4, 10),
                'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
                'iterations': trial.suggest_int('iterations', 50, 550),
                'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 9),
                'border_count': trial.suggest_categorical('border_count', [32, 64, 128, 254]),
                'task_type': 'GPU' if device == 'cuda' else 'CPU',
                'random_state': random_state,
                'verbose': False
            }
            clf = CatBoostClassifier(**params)
            clf.fit(X_train_scaled, y_train, eval_set=(X_test_scaled, y_test), 
                    early_stopping_rounds=10, verbose=False)
        
        y_pred = clf.predict(X_test_scaled)
        return accuracy_score(y_test, y_pred)

    start_time = time.time()
    print(f"Starting Bayesian Optimization for {classifier_type.upper()}...")

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100)

    end_time = time.time()
    print(f"Finished Bayesian Optimization tuning. Time taken: {end_time - start_time:.2f} seconds")

    best_params = study.best_params
    best_accuracy = study.best_value

    return best_params, best_accuracy

# Main execution
random_state = 42

# Assuming X and y are your feature matrix and target vector
# X = ...
# y = ...

for classifier_type in ['xgboost']:
    print(f"\nEvaluating {classifier_type.upper()}")
    
    print("Genetic Algorithm Optimization")
    ga_best_params, ga_best_accuracy = genetic_algorithm(X, y, classifier_type, random_state=random_state)
    print(f"\nBest parameters for {classifier_type.upper()} (Genetic Algorithm):")
    for param, value in ga_best_params.items():
        print(f"{param}: {value}")
    print(f"Best accuracy: {ga_best_accuracy:.4f}")
    
    print("\nBayesian Optimization")
    bo_best_params, bo_best_accuracy = bayesian_optimization(X, y, classifier_type, random_state=random_state)
    print(f"\nBest parameters for {classifier_type.upper()} (Bayesian Optimization):")
    for param, value in bo_best_params.items():
        print(f"{param}: {value}")
    print(f"Best accuracy: {bo_best_accuracy:.4f}")

print(f"\nNumber of training samples: 840")
print(f"Number of testing samples: 360")
print(f"Total samples: 1200")


Evaluating XGBOOST
Genetic Algorithm Optimization
Starting Genetic Algorithm optimization for XGBOOST...
gen	nevals
0  	50    
1  	29    
2  	36    
3  	23    
4  	35    
5  	29    
6  	29    
7  	35    
8  	36    
9  	32    


[I 2024-11-04 00:53:14,927] A new study created in memory with name: no-name-4423fa4f-adae-4149-9da3-2a2e3913fbcb


10 	31    
Finished Genetic Algorithm tuning. Time taken: 211.80 seconds

Best parameters for XGBOOST (Genetic Algorithm):
max_depth: 2
learning_rate: 0.2
n_estimators: 500
subsample: 0.8
colsample_bytree: 0.7
Best accuracy: 0.9611

Bayesian Optimization
Starting Bayesian Optimization for XGBOOST...


[I 2024-11-04 00:53:15,868] Trial 0 finished with value: 0.95 and parameters: {'max_depth': 7, 'learning_rate': 0.03354925118829631, 'n_estimators': 521, 'subsample': 0.7593081143240401, 'colsample_bytree': 0.6350385864774327}. Best is trial 0 with value: 0.95.
[I 2024-11-04 00:53:16,960] Trial 1 finished with value: 0.9388888888888889 and parameters: {'max_depth': 8, 'learning_rate': 0.010908491107232394, 'n_estimators': 289, 'subsample': 0.7655945181856924, 'colsample_bytree': 0.8239364080913907}. Best is trial 0 with value: 0.95.
[I 2024-11-04 00:53:17,779] Trial 2 finished with value: 0.9472222222222222 and parameters: {'max_depth': 5, 'learning_rate': 0.020807164454014725, 'n_estimators': 203, 'subsample': 0.6339728494752142, 'colsample_bytree': 0.8053107782145753}. Best is trial 0 with value: 0.95.
[I 2024-11-04 00:53:18,309] Trial 3 finished with value: 0.95 and parameters: {'max_depth': 7, 'learning_rate': 0.24646854578377822, 'n_estimators': 503, 'subsample': 0.689105254084607

Finished Bayesian Optimization tuning. Time taken: 76.81 seconds

Best parameters for XGBOOST (Bayesian Optimization):
max_depth: 6
learning_rate: 0.03758882599317193
n_estimators: 287
subsample: 0.6926137439560578
colsample_bytree: 0.6892905257497403
Best accuracy: 0.9528

Number of training samples: 840
Number of testing samples: 360
Total samples: 1200


In [42]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from deap import base, creator, tools, algorithms
import optuna
import random
import time
import warnings

warnings.filterwarnings("ignore")


param_space = {
    'n_estimators': list(range(10, 501, 10)),
    'max_depth': [None] + list(range(5, 51, 5)),
    'min_samples_split': list(range(2, 21)),
    'min_samples_leaf': list(range(1, 11))
}

def prepare_data(X, y, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=360, train_size=840, stratify=y, random_state=random_state)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, y_train, y_test

def genetic_algorithm(X, y, random_state=42):
    X_train_scaled, X_test_scaled, y_train, y_test = prepare_data(X, y, random_state)

    def evaluate(individual):
        clf = RandomForestClassifier(
            n_estimators=individual[0],
            max_depth=individual[1],
            min_samples_split=individual[2],
            min_samples_leaf=individual[3],
            random_state=random_state,
            n_jobs=-1
        )
        
        try:
            clf.fit(X_train_scaled, y_train)
            y_pred = clf.predict(X_test_scaled)
            return accuracy_score(y_test, y_pred),
        except Exception as e:
            print(f"Error during evaluation: {e}")
            return 0,

    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)
    toolbox = base.Toolbox()
    toolbox.register("attr_n_estimators", random.choice, param_space['n_estimators'])
    toolbox.register("attr_max_depth", random.choice, param_space['max_depth'])
    toolbox.register("attr_min_samples_split", random.choice, param_space['min_samples_split'])
    toolbox.register("attr_min_samples_leaf", random.choice, param_space['min_samples_leaf'])
    toolbox.register("individual", tools.initCycle, creator.Individual,
                     (toolbox.attr_n_estimators, toolbox.attr_max_depth, 
                      toolbox.attr_min_samples_split, toolbox.attr_min_samples_leaf), n=1)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", evaluate)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutUniformInt, low=1, up=500, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)

    population = toolbox.population(n=50)
    ngen = 10

    start_time = time.time()
    print("Starting Genetic Algorithm optimization for Random Forest...")

    result, log = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=ngen, verbose=True)

    end_time = time.time()
    print(f"Finished Genetic Algorithm tuning. Time taken: {end_time - start_time:.2f} seconds")

    best_individual = tools.selBest(result, k=1)[0]
    best_params = {
        'n_estimators': best_individual[0],
        'max_depth': best_individual[1],
        'min_samples_split': best_individual[2],
        'min_samples_leaf': best_individual[3]
    }
    best_accuracy = best_individual.fitness.values[0]

    return best_params, best_accuracy

def bayesian_optimization(X, y, random_state=42):
    X_train_scaled, X_test_scaled, y_train, y_test = prepare_data(X, y, random_state)

    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 10, 500),
            'max_depth': trial.suggest_int('max_depth', 1, 50),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
            'random_state': random_state,
            'n_jobs': -1
        }
        
        if params['max_depth'] == 1:
            params['max_depth'] = None
        
        clf = RandomForestClassifier(**params)
        clf.fit(X_train_scaled, y_train)
        y_pred = clf.predict(X_test_scaled)
        return accuracy_score(y_test, y_pred)

    start_time = time.time()
    print("Starting Bayesian Optimization for Random Forest...")

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100)

    end_time = time.time()
    print(f"Finished Bayesian Optimization tuning. Time taken: {end_time - start_time:.2f} seconds")

    best_params = study.best_params
    if best_params['max_depth'] == 1:
        best_params['max_depth'] = None
    best_accuracy = study.best_value

    return best_params, best_accuracy

# Main execution
random_state = 42

# Assuming X and y are your feature matrix and target vector
# X = ...
# y = ...

print("Evaluating Random Forest with Genetic Algorithm Optimization")
ga_best_params, ga_best_accuracy = genetic_algorithm(X, y, random_state=random_state)
print("\nBest parameters for Random Forest (Genetic Algorithm):")
for param, value in ga_best_params.items():
    print(f"{param}: {value}")
print(f"Best accuracy: {ga_best_accuracy:.4f}")

print("\nEvaluating Random Forest with Bayesian Optimization")
bo_best_params, bo_best_accuracy = bayesian_optimization(X, y, random_state=random_state)
print("\nBest parameters for Random Forest (Bayesian Optimization):")
for param, value in bo_best_params.items():
    print(f"{param}: {value}")
print(f"Best accuracy: {bo_best_accuracy:.4f}")

print(f"\nNumber of training samples: 840")
print(f"Number of testing samples: 360")
print(f"Total samples: 1200")

Evaluating Random Forest with Genetic Algorithm Optimization
Starting Genetic Algorithm optimization for Random Forest...
gen	nevals
0  	50    
1  	35    
2  	35    
3  	26    
4  	32    
5  	34    
6  	25    
7  	26    
Error during evaluation: The 'min_samples_split' parameter of RandomForestClassifier must be an int in the range [2, inf) or a float in the range (0.0, 1.0]. Got 1 instead.
8  	36    
9  	29    


[I 2024-11-04 01:01:02,311] A new study created in memory with name: no-name-2ffe2ab6-a6c8-4c4b-a5bb-37787ec96843


10 	31    
Finished Genetic Algorithm tuning. Time taken: 390.50 seconds

Best parameters for Random Forest (Genetic Algorithm):
n_estimators: 310
max_depth: 106
min_samples_split: 6
min_samples_leaf: 2
Best accuracy: 0.9472

Evaluating Random Forest with Bayesian Optimization
Starting Bayesian Optimization for Random Forest...


[I 2024-11-04 01:01:04,026] Trial 0 finished with value: 0.9333333333333333 and parameters: {'n_estimators': 495, 'max_depth': 32, 'min_samples_split': 12, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.9333333333333333.
[I 2024-11-04 01:01:05,059] Trial 1 finished with value: 0.9388888888888889 and parameters: {'n_estimators': 283, 'max_depth': 47, 'min_samples_split': 4, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.9388888888888889.
[I 2024-11-04 01:01:05,174] Trial 2 finished with value: 0.8805555555555555 and parameters: {'n_estimators': 15, 'max_depth': 2, 'min_samples_split': 11, 'min_samples_leaf': 6}. Best is trial 1 with value: 0.9388888888888889.
[I 2024-11-04 01:01:06,263] Trial 3 finished with value: 0.9277777777777778 and parameters: {'n_estimators': 303, 'max_depth': 39, 'min_samples_split': 9, 'min_samples_leaf': 9}. Best is trial 1 with value: 0.9388888888888889.
[I 2024-11-04 01:01:06,937] Trial 4 finished with value: 0.9361111111111111 and parameters: 

Finished Bayesian Optimization tuning. Time taken: 119.99 seconds

Best parameters for Random Forest (Bayesian Optimization):
n_estimators: 345
max_depth: 24
min_samples_split: 17
min_samples_leaf: 1
Best accuracy: 0.9500

Number of training samples: 840
Number of testing samples: 360
Total samples: 1200


In [43]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from deap import base, creator, tools, algorithms
import optuna
import random
import time
import warnings

warnings.filterwarnings("ignore")

# Define the parameter spaces for Logistic Regression
param_space = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
    'max_iter': [100, 200, 500, 1000, 2000],
    'penalty': ['l2', 'l1', 'elasticnet', None]
}

def safe_choice(options, index):
    return options[index % len(options)]

def prepare_data(X, y, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=360, train_size=840, stratify=y, random_state=random_state)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, y_train, y_test

def genetic_algorithm(X, y, random_state=42):
    X_train_scaled, X_test_scaled, y_train, y_test = prepare_data(X, y, random_state)

    def evaluate(individual):
        clf = LogisticRegression(
            C=safe_choice(param_space['C'], individual[0]),
            solver=safe_choice(param_space['solver'], individual[1]),
            max_iter=safe_choice(param_space['max_iter'], individual[2]),
            penalty=safe_choice(param_space['penalty'], individual[3]),
            random_state=random_state,
            n_jobs=-1
        )
        
        try:
            clf.fit(X_train_scaled, y_train)
            y_pred = clf.predict(X_test_scaled)
            return accuracy_score(y_test, y_pred),
        except Exception as e:
            print(f"Error during evaluation: {e}")
            return 0,

    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)
    toolbox = base.Toolbox()
    toolbox.register("attr_C", random.randint, 0, len(param_space['C'])-1)
    toolbox.register("attr_solver", random.randint, 0, len(param_space['solver'])-1)
    toolbox.register("attr_max_iter", random.randint, 0, len(param_space['max_iter'])-1)
    toolbox.register("attr_penalty", random.randint, 0, len(param_space['penalty'])-1)
    toolbox.register("individual", tools.initCycle, creator.Individual,
                     (toolbox.attr_C, toolbox.attr_solver, 
                      toolbox.attr_max_iter, toolbox.attr_penalty), n=1)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", evaluate)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutUniformInt, low=0, up=10, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)

    population = toolbox.population(n=24)
    ngen = 5

    start_time = time.time()
    print("Starting Genetic Algorithm optimization for Logistic Regression...")

    result, log = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=ngen, verbose=True)

    end_time = time.time()
    print(f"Finished Genetic Algorithm tuning. Time taken: {end_time - start_time:.2f} seconds")

    best_individual = tools.selBest(result, k=1)[0]
    best_params = {
        'C': safe_choice(param_space['C'], best_individual[0]),
        'solver': safe_choice(param_space['solver'], best_individual[1]),
        'max_iter': safe_choice(param_space['max_iter'], best_individual[2]),
        'penalty': safe_choice(param_space['penalty'], best_individual[3]),
    }
    best_accuracy = best_individual.fitness.values[0]

    return best_params, best_accuracy

def bayesian_optimization(X, y, random_state=42):
    X_train_scaled, X_test_scaled, y_train, y_test = prepare_data(X, y, random_state)

    def objective(trial):
        params = {
            'C': trial.suggest_categorical('C', param_space['C']),
            'solver': trial.suggest_categorical('solver', param_space['solver']),
            'max_iter': trial.suggest_categorical('max_iter', param_space['max_iter']),
            'penalty': trial.suggest_categorical('penalty', param_space['penalty']),
            'random_state': random_state,
            'n_jobs': -1
        }
        
        # Handle the elasticnet case
        if params['penalty'] == 'elasticnet':
            params['l1_ratio'] = trial.suggest_uniform('l1_ratio', 0, 1)
        
        # Handle solver-penalty incompatibilities
        if params['solver'] == 'liblinear' and params['penalty'] not in ['l1', 'l2']:
            params['penalty'] = 'l2'
        elif params['solver'] in ['newton-cg', 'sag', 'lbfgs'] and params['penalty'] == 'l1':
            params['penalty'] = 'l2'
        
        try:
            clf = LogisticRegression(**params)
            clf.fit(X_train_scaled, y_train)
            y_pred = clf.predict(X_test_scaled)
            return accuracy_score(y_test, y_pred)
        except Exception as e:
            print(f"Error during evaluation: {e}")
            return 0  # Return a low score for failed trials

    start_time = time.time()
    print("Starting Bayesian Optimization for Logistic Regression...")

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100)

    end_time = time.time()
    print(f"Finished Bayesian Optimization tuning. Time taken: {end_time - start_time:.2f} seconds")

    best_params = study.best_params
    best_accuracy = study.best_value

    return best_params, best_accuracy

# Main execution
random_state = 42

# Assuming X and y are your feature matrix and target vector
# X = ...
# y = ...

print("Evaluating Logistic Regression with Genetic Algorithm Optimization")
ga_best_params, ga_best_accuracy = genetic_algorithm(X, y, random_state=random_state)
print("\nBest parameters for Logistic Regression (Genetic Algorithm):")
for param, value in ga_best_params.items():
    print(f"{param}: {value}")
print(f"Best accuracy: {ga_best_accuracy:.4f}")

print("\nEvaluating Logistic Regression with Bayesian Optimization")
bo_best_params, bo_best_accuracy = bayesian_optimization(X, y, random_state=random_state)
print("\nBest parameters for Logistic Regression (Bayesian Optimization):")
for param, value in bo_best_params.items():
    print(f"{param}: {value}")
print(f"Best accuracy: {bo_best_accuracy:.4f}")

print(f"\nNumber of training samples: 840")
print(f"Number of testing samples: 360")
print(f"Total samples: 1200")

Evaluating Logistic Regression with Genetic Algorithm Optimization
Starting Genetic Algorithm optimization for Logistic Regression...
Error during evaluation: Solver sag supports only 'l2' or None penalties, got l1 penalty.
Error during evaluation: penalty=None is not supported for the liblinear solver


/home/f20210934/.local/lib/python3.10/site-packages/sklearn/linear_model/_glm/_newton_solver.py:497: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=1.38642e-17): result may not be accurate.
  warnings.warn(


Error during evaluation: Solver newton-cg supports only 'l2' or None penalties, got elasticnet penalty.
Error during evaluation: Solver sag supports only 'l2' or None penalties, got l1 penalty.
Error during evaluation: Solver newton-cg supports only 'l2' or None penalties, got elasticnet penalty.
Error during evaluation: Solver sag supports only 'l2' or None penalties, got l1 penalty.
Error during evaluation: Solver newton-cholesky supports only 'l2' or None penalties, got elasticnet penalty.
Error during evaluation: l1_ratio must be specified when penalty is elasticnet.
Error during evaluation: Solver newton-cholesky supports only 'l2' or None penalties, got l1 penalty.
Error during evaluation: Solver newton-cg supports only 'l2' or None penalties, got l1 penalty.
Error during evaluation: Solver newton-cg supports only 'l2' or None penalties, got elasticnet penalty.
Error during evaluation: Solver lbfgs supports only 'l2' or None penalties, got elasticnet penalty.
Error during evaluat

/home/f20210934/.local/lib/python3.10/site-packages/sklearn/linear_model/_glm/_newton_solver.py:497: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=1.38642e-17): result may not be accurate.
  warnings.warn(


gen	nevals
0  	24    
Error during evaluation: Solver newton-cholesky supports only 'l2' or None penalties, got l1 penalty.
Error during evaluation: Solver newton-cholesky supports only 'l2' or None penalties, got l1 penalty.
Error during evaluation: Solver newton-cg supports only 'l2' or None penalties, got l1 penalty.
Error during evaluation: Solver newton-cholesky supports only 'l2' or None penalties, got l1 penalty.


...

Error during evaluation: Solver newton-cg supports only 'l2' or None penalties, got elasticnet penalty.


[I 2024-11-04 01:06:35,816] Trial 40 finished with value: 0.95 and parameters: {'C': 1, 'solver': 'lbfgs', 'max_iter': 100, 'penalty': None}. Best is trial 13 with value: 0.9805555555555555.
[I 2024-11-04 01:06:35,978] Trial 41 finished with value: 0.9805555555555555 and parameters: {'C': 1, 'solver': 'liblinear', 'max_iter': 2000, 'penalty': 'l2'}. Best is trial 13 with value: 0.9805555555555555.
[I 2024-11-04 01:06:36,137] Trial 42 finished with value: 0.9805555555555555 and parameters: {'C': 1, 'solver': 'liblinear', 'max_iter': 2000, 'penalty': 'l2'}. Best is trial 13 with value: 0.9805555555555555.
[I 2024-11-04 01:06:36,302] Trial 43 finished with value: 0.9805555555555555 and parameters: {'C': 1, 'solver': 'liblinear', 'max_iter': 2000, 'penalty': 'elasticnet', 'l1_ratio': 0.006405069042841327}. Best is trial 13 with value: 0.9805555555555555.
[I 2024-11-04 01:06:46,153] Trial 44 finished with value: 0.9722222222222222 and parameters: {'C': 10, 'solver': 'sag', 'max_iter': 2000,

Error during evaluation: Solver newton-cg supports only 'l2' or None penalties, got elasticnet penalty.


[I 2024-11-04 01:07:12,870] Trial 69 finished with value: 0.9722222222222222 and parameters: {'C': 10, 'solver': 'saga', 'max_iter': 2000, 'penalty': None}. Best is trial 13 with value: 0.9805555555555555.
[I 2024-11-04 01:07:13,007] Trial 70 finished with value: 0.9805555555555555 and parameters: {'C': 0.1, 'solver': 'liblinear', 'max_iter': 500, 'penalty': 'l2'}. Best is trial 13 with value: 0.9805555555555555.
[I 2024-11-04 01:07:13,139] Trial 71 finished with value: 0.9805555555555555 and parameters: {'C': 0.1, 'solver': 'liblinear', 'max_iter': 2000, 'penalty': 'l2'}. Best is trial 13 with value: 0.9805555555555555.
[I 2024-11-04 01:07:13,271] Trial 72 finished with value: 0.9805555555555555 and parameters: {'C': 0.1, 'solver': 'liblinear', 'max_iter': 2000, 'penalty': 'l2'}. Best is trial 13 with value: 0.9805555555555555.
[I 2024-11-04 01:07:13,402] Trial 73 finished with value: 0.9805555555555555 and parameters: {'C': 0.1, 'solver': 'liblinear', 'max_iter': 2000, 'penalty': 'l2

Finished Bayesian Optimization tuning. Time taken: 87.88 seconds

Best parameters for Logistic Regression (Bayesian Optimization):
C: 0.1
solver: liblinear
max_iter: 500
penalty: l2
Best accuracy: 0.9806

Number of training samples: 840
Number of testing samples: 360
Total samples: 1200


In [44]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from deap import base, creator, tools, algorithms
import optuna
import random
import time
import warnings

warnings.filterwarnings("ignore")

# Define the parameter spaces for SVM and KNN
param_space = {
    'svm': {
        'C': [0.1, 1, 10, 100],
        'kernel': ['rbf', 'linear', 'poly', 'sigmoid'],
        'gamma': ['scale', 'auto'] + [0.0001, 0.001, 0.01, 0.1, 1],
        'degree': [2, 3, 4, 5]  # Only used when kernel='poly'
    },
    'knn': {
        'n_neighbors': list(range(1, 31)),
        'weights': ['uniform', 'distance'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
        'p': [1, 2]  # 1 for manhattan_distance, 2 for euclidean_distance
    }
}

def prepare_data(X, y, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=360, train_size=840, stratify=y, random_state=random_state)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, y_train, y_test

def genetic_algorithm(X, y, classifier_type, random_state=42):
    X_train_scaled, X_test_scaled, y_train, y_test = prepare_data(X, y, random_state)

    def evaluate(individual):
        if classifier_type == 'svm':
            clf = SVC(
                C=individual[0],
                kernel=individual[1],
                gamma=individual[2],
                degree=individual[3] if individual[1] == 'poly' else 3,
                random_state=random_state
            )
        else:  # KNN
            clf = KNeighborsClassifier(
                n_neighbors=individual[0],
                weights=individual[1],
                algorithm=individual[2],
                p=individual[3]
            )
        
        try:
            clf.fit(X_train_scaled, y_train)
            y_pred = clf.predict(X_test_scaled)
            return accuracy_score(y_test, y_pred),
        except Exception as e:
            print(f"Error during evaluation: {e}")
            return 0,

    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)
    toolbox = base.Toolbox()

    if classifier_type == 'svm':
        toolbox.register("attr_C", random.choice, param_space['svm']['C'])
        toolbox.register("attr_kernel", random.choice, param_space['svm']['kernel'])
        toolbox.register("attr_gamma", random.choice, param_space['svm']['gamma'])
        toolbox.register("attr_degree", random.choice, param_space['svm']['degree'])
    else:  # KNN
        toolbox.register("attr_n_neighbors", random.choice, param_space['knn']['n_neighbors'])
        toolbox.register("attr_weights", random.choice, param_space['knn']['weights'])
        toolbox.register("attr_algorithm", random.choice, param_space['knn']['algorithm'])
        toolbox.register("attr_p", random.choice, param_space['knn']['p'])

    toolbox.register("individual", tools.initCycle, creator.Individual,
                     (toolbox.attr_C, toolbox.attr_kernel, toolbox.attr_gamma, toolbox.attr_degree) if classifier_type == 'svm'
                     else (toolbox.attr_n_neighbors, toolbox.attr_weights, toolbox.attr_algorithm, toolbox.attr_p),
                     n=1)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", evaluate)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutUniformInt, low=0, up=10, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)

    population = toolbox.population(n=50)
    ngen = 10

    start_time = time.time()
    print(f"Starting Genetic Algorithm optimization for {classifier_type.upper()}...")

    result, log = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=ngen, verbose=True)

    end_time = time.time()
    print(f"Finished Genetic Algorithm tuning. Time taken: {end_time - start_time:.2f} seconds")

    best_individual = tools.selBest(result, k=1)[0]
    if classifier_type == 'svm':
        best_params = {
            'C': best_individual[0],
            'kernel': best_individual[1],
            'gamma': best_individual[2],
            'degree': best_individual[3] if best_individual[1] == 'poly' else 3
        }
    else:  # KNN
        best_params = {
            'n_neighbors': best_individual[0],
            'weights': best_individual[1],
            'algorithm': best_individual[2],
            'p': best_individual[3]
        }
    best_accuracy = best_individual.fitness.values[0]

    return best_params, best_accuracy

def bayesian_optimization(X, y, classifier_type, random_state=42):
    X_train_scaled, X_test_scaled, y_train, y_test = prepare_data(X, y, random_state)

    def objective(trial):
        if classifier_type == 'svm':
            params = {
                'C': trial.suggest_categorical('C', param_space['svm']['C']),
                'kernel': trial.suggest_categorical('kernel', param_space['svm']['kernel']),
                'gamma': trial.suggest_categorical('gamma', param_space['svm']['gamma']),
                'random_state': random_state
            }
            if params['kernel'] == 'poly':
                params['degree'] = trial.suggest_int('degree', 2, 5)
            clf = SVC(**params)
        else:  # KNN
            params = {
                'n_neighbors': trial.suggest_int('n_neighbors', 1, 30),
                'weights': trial.suggest_categorical('weights', param_space['knn']['weights']),
                'algorithm': trial.suggest_categorical('algorithm', param_space['knn']['algorithm']),
                'p': trial.suggest_categorical('p', param_space['knn']['p'])
            }
            clf = KNeighborsClassifier(**params)
        
        clf.fit(X_train_scaled, y_train)
        y_pred = clf.predict(X_test_scaled)
        return accuracy_score(y_test, y_pred)

    start_time = time.time()
    print(f"Starting Bayesian Optimization for {classifier_type.upper()}...")

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100)

    end_time = time.time()
    print(f"Finished Bayesian Optimization tuning. Time taken: {end_time - start_time:.2f} seconds")

    best_params = study.best_params
    best_accuracy = study.best_value

    return best_params, best_accuracy

# Main execution
random_state = 42

# Assuming X and y are your feature matrix and target vector
# X = ...
# y = ...

for classifier_type in ['svm', 'knn']:
    print(f"\nEvaluating {classifier_type.upper()}")
    
    print("Genetic Algorithm Optimization")
    ga_best_params, ga_best_accuracy = genetic_algorithm(X, y, classifier_type, random_state=random_state)
    print(f"\nBest parameters for {classifier_type.upper()} (Genetic Algorithm):")
    for param, value in ga_best_params.items():
        print(f"{param}: {value}")
    print(f"Best accuracy: {ga_best_accuracy:.4f}")
    
    print("\nBayesian Optimization")
    bo_best_params, bo_best_accuracy = bayesian_optimization(X, y, classifier_type, random_state=random_state)
    print(f"\nBest parameters for {classifier_type.upper()} (Bayesian Optimization):")
    for param, value in bo_best_params.items():
        print(f"{param}: {value}")
    print(f"Best accuracy: {bo_best_accuracy:.4f}")

print(f"\nNumber of training samples: 840")
print(f"Number of testing samples: 360")
print(f"Total samples: 1200")


Evaluating SVM
Genetic Algorithm Optimization
Starting Genetic Algorithm optimization for SVM...
gen	nevals
0  	50    
Error during evaluation: The 'kernel' parameter of SVC must be a str among {'linear', 'precomputed', 'sigmoid', 'poly', 'rbf'} or a callable. Got 10 instead.
Error during evaluation: The 'kernel' parameter of SVC must be a str among {'linear', 'precomputed', 'sigmoid', 'poly', 'rbf'} or a callable. Got 1 instead.
1  	29    
Error during evaluation: The 'kernel' parameter of SVC must be a str among {'linear', 'precomputed', 'sigmoid', 'poly', 'rbf'} or a callable. Got 5 instead.
Error during evaluation: The 'C' parameter of SVC must be a float in the range (0.0, inf). Got 0 instead.
Error during evaluation: The 'kernel' parameter of SVC must be a str among {'linear', 'precomputed', 'sigmoid', 'poly', 'rbf'} or a callable. Got 10 instead.
2  	32    
Error during evaluation: The 'kernel' parameter of SVC must be a str among {'linear', 'precomputed', 'sigmoid', 'poly', 'r

[I 2024-11-04 01:07:54,213] A new study created in memory with name: no-name-acdf771b-ac44-4608-b446-ce406e3a6c52
[I 2024-11-04 01:07:54,339] Trial 0 finished with value: 0.8388888888888889 and parameters: {'C': 0.1, 'kernel': 'poly', 'gamma': 'auto', 'degree': 3}. Best is trial 0 with value: 0.8388888888888889.


10 	25    
Finished Genetic Algorithm tuning. Time taken: 21.95 seconds

Best parameters for SVM (Genetic Algorithm):
C: 100
kernel: sigmoid
gamma: 0.0001
degree: 3
Best accuracy: 0.9778

Bayesian Optimization
Starting Bayesian Optimization for SVM...


[I 2024-11-04 01:07:54,625] Trial 1 finished with value: 0.55 and parameters: {'C': 10, 'kernel': 'rbf', 'gamma': 0.1}. Best is trial 0 with value: 0.8388888888888889.
[I 2024-11-04 01:07:54,719] Trial 2 finished with value: 0.95 and parameters: {'C': 0.1, 'kernel': 'poly', 'gamma': 0.1, 'degree': 4}. Best is trial 2 with value: 0.95.
[I 2024-11-04 01:07:54,821] Trial 3 finished with value: 0.9194444444444444 and parameters: {'C': 100, 'kernel': 'poly', 'gamma': 1, 'degree': 5}. Best is trial 2 with value: 0.95.
[I 2024-11-04 01:07:54,880] Trial 4 finished with value: 0.9638888888888889 and parameters: {'C': 100, 'kernel': 'poly', 'gamma': 1, 'degree': 2}. Best is trial 4 with value: 0.9638888888888889.
[I 2024-11-04 01:07:55,128] Trial 5 finished with value: 0.9527777777777777 and parameters: {'C': 1, 'kernel': 'rbf', 'gamma': 0.01}. Best is trial 4 with value: 0.9638888888888889.
[I 2024-11-04 01:07:55,204] Trial 6 finished with value: 0.7638888888888888 and parameters: {'C': 100, 'k

Finished Bayesian Optimization tuning. Time taken: 9.29 seconds

Best parameters for SVM (Bayesian Optimization):
C: 100
kernel: rbf
gamma: 0.001
Best accuracy: 0.9667

Evaluating KNN
Genetic Algorithm Optimization
Starting Genetic Algorithm optimization for KNN...
gen	nevals
0  	50    
Error during evaluation: The 'algorithm' parameter of KNeighborsClassifier must be a str among {'kd_tree', 'brute', 'ball_tree', 'auto'}. Got 5 instead.
Error during evaluation: The 'weights' parameter of KNeighborsClassifier must be a str among {'distance', 'uniform'}, a callable or None. Got 1 instead.
Error during evaluation: The 'weights' parameter of KNeighborsClassifier must be a str among {'distance', 'uniform'}, a callable or None. Got 2 instead.
Error during evaluation: The 'weights' parameter of KNeighborsClassifier must be a str among {'distance', 'uniform'}, a callable or None. Got 2 instead.
Error during evaluation: The 'algorithm' parameter of KNeighborsClassifier must be a str among {'kd_

[I 2024-11-04 01:11:55,237] A new study created in memory with name: no-name-7bfa729c-bea8-4ff8-a512-106cc4ab83ab


10 	29    
Finished Genetic Algorithm tuning. Time taken: 231.69 seconds

Best parameters for KNN (Genetic Algorithm):
n_neighbors: 2
weights: distance
algorithm: auto
p: 3
Best accuracy: 0.9444

Bayesian Optimization
Starting Bayesian Optimization for KNN...


[I 2024-11-04 01:11:55,613] Trial 0 finished with value: 0.8972222222222223 and parameters: {'n_neighbors': 29, 'weights': 'distance', 'algorithm': 'kd_tree', 'p': 2}. Best is trial 0 with value: 0.8972222222222223.
[I 2024-11-04 01:11:55,716] Trial 1 finished with value: 0.9083333333333333 and parameters: {'n_neighbors': 20, 'weights': 'distance', 'algorithm': 'brute', 'p': 1}. Best is trial 1 with value: 0.9083333333333333.
[I 2024-11-04 01:11:55,812] Trial 2 finished with value: 0.9111111111111111 and parameters: {'n_neighbors': 18, 'weights': 'distance', 'algorithm': 'brute', 'p': 1}. Best is trial 2 with value: 0.9111111111111111.
[I 2024-11-04 01:11:56,192] Trial 3 finished with value: 0.8916666666666667 and parameters: {'n_neighbors': 24, 'weights': 'uniform', 'algorithm': 'kd_tree', 'p': 1}. Best is trial 2 with value: 0.9111111111111111.
[I 2024-11-04 01:11:56,221] Trial 4 finished with value: 0.9305555555555556 and parameters: {'n_neighbors': 5, 'weights': 'uniform', 'algorit

Finished Bayesian Optimization tuning. Time taken: 26.82 seconds

Best parameters for KNN (Bayesian Optimization):
n_neighbors: 1
weights: uniform
algorithm: ball_tree
p: 2
Best accuracy: 0.9389

Number of training samples: 840
Number of testing samples: 360
Total samples: 1200
